# Entanglement Classifier Training Trial

## Importing Dependencies

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sympy.physics.quantum import TensorProduct

from components.system import System
from components.connection import ConnectionFactory, Connection
from components.model import Model
from components.connection_functions import DrivenCascadeFunction, EnergyExchangeFunction
from components.state_generator import generateMixedBatch, generatePureBatch, generateMixedBatchUsingPure

## Generating Training Data

In [2]:
samples = 2000
test_size = 0.5

In [3]:
x_pure = generatePureBatch(int(samples/2))
y_pure = np.ones(int(samples/2))
x_mixed = generateMixedBatchUsingPure(int(samples/2))
y_mixed = np.zeros(int(samples/2))

In [4]:
x = np.concatenate([x_pure, x_mixed])
y = np.concatenate([y_pure, y_mixed])

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42, shuffle=True)

## Generate Reservoir Measurements

### Setting up the system

In [6]:
reservoir_nodes=2
system_nodes=1

In [7]:
"""
Defining System setup
"""

system_state = np.array([[1, 0], [0, 0]])
system_node_list = [0]

if len(system_node_list) != system_nodes:
    raise Exception

system_connections = {}

system = System(
    init_quantum_state=system_state, nodes=system_node_list, connections=system_connections
)

### Setting up the interface

In [8]:
"""
Defining Interface setup
"""

interfaceFactory = ConnectionFactory(DrivenCascadeFunction, gamma_1=[8,12], gamma_2=[8,12])

### Setting up the reservoir

In [9]:
"""
Defining Reservoir setup
"""

reservoirFactory = ConnectionFactory(EnergyExchangeFunction, is_hamiltonian=True, J=[0.5,1.5])

### Setting up the model

In [10]:
model = Model()
model.setSystem(system)
model.setReservoirConnectionFac(reservoirFactory)
model.setInterfaceConectionFac(interfaceFactory)
model.generateReservoir(reservoir_nodes, init_quantum_state=0)
model.generateInterface(connection_rate=.5)
model.setRunDuration(0.1)
model.setRunResolution(0.01)

### Running the model

In [11]:
d_train = []
d_test = []
measurement_iteration = -1

for state in x_train:
    model.system.init_quantum_state = state
    model.run()
    d_train.append(np.real(list(model.modelLog.excitation_time_log.values())[measurement_iteration]))

for state in x_test:
    model.system.init_quantum_state = state
    model.run()
    d_test.append(np.real(list(model.modelLog.excitation_time_log.values())[measurement_iteration]))

## Classical Processing

In [12]:
sgd = SGDClassifier()
sgd.fit(d_train, y_train)

SGDClassifier()

In [13]:
sgd.score(d_test, y_test)

0.762

In [16]:
y_pred = sgd.predict(d_test)

In [14]:
from sklearn.metrics import confusion_matrix

In [30]:
wrong = []

for i in range(len(y_test)):
    if y_test[i] != y_pred[i]:
        if y_test[i] == 1:
            wrong.append([x_test[i], np.trace(np.matmul(x_test[i], x_test[i])), y_test[i]])

In [34]:
wrong

[[array([[0.94967265, 0.21861954],
         [0.21861954, 0.05032735]]),
  0.9999999999999998,
  1.0],
 [array([[0.02424401, 0.15380584],
         [0.15380584, 0.97575599]]),
  0.9999999999999996,
  1.0],
 [array([[0.96913106, 0.17296257],
         [0.17296257, 0.03086894]]),
  1.0000000000000002,
  1.0],
 [array([[0.9605127 , 0.19475126],
         [0.19475126, 0.0394873 ]]),
  1.0000000000000004,
  1.0],
 [array([[0.98305551, 0.12906345],
         [0.12906345, 0.01694449]]),
  0.9999999999999996,
  1.0],
 [array([[9.12501677e-04, 3.01938573e-02],
         [3.01938573e-02, 9.99087498e-01]]),
  1.0000000000000004,
  1.0],
 [array([[0.96353116, 0.18745363],
         [0.18745363, 0.03646884]]),
  1.0000000000000002,
  1.0],
 [array([[7.39850363e-04, 2.71901266e-02],
         [2.71901266e-02, 9.99260150e-01]]),
  0.9999999999999997,
  1.0],
 [array([[0.95571377, 0.2057303 ],
         [0.2057303 , 0.04428623]]),
  0.9999999999999998,
  1.0],
 [array([[0.00869957, 0.09286489],
         [0.092